# Microsoft Agent Framework + Llama Stack Integration

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meta-llama/llama-stack/blob/main/docs/notebooks/autogen/microsoft_agent_framework_llama_stack_integration.ipynb)

## Overview

This notebook demonstrates how to use **Microsoft Agent Framework** (successor to AutoGen) with **Llama Stack** as the backend.

> **Note:** This notebook uses Microsoft Agent Framework, which replaces AutoGen. For the migration guide, see: [Microsoft Agent Framework Migration Guide](https://learn.microsoft.com/en-us/agent-framework/migration-guide/from-autogen/)

### Use Cases Covered:
1. **Two-Agent Conversation** - Teams working together on tasks
2. **Code Generation & Execution** - Agents generating and running code
3. **Group Chat** - Multiple specialists collaborating  
4. **Advanced Termination** - Stopping conditions

---

## Prerequisites

```bash
# Install Microsoft Agent Framework
pip install agent-framework

# Llama Stack should already be running
# Default: http://localhost:8321
```

**Migration Note:** If you're migrating from AutoGen, the main changes are:
- Package: `autogen-*` → `agent-framework`
- Client: `OpenAIChatCompletionClient` → `OpenAIResponsesClient`
- Agent: `AssistantAgent` → `ChatAgent`
- Team: `RoundRobinGroupChat` → `SequentialBuilder`

In [19]:
# Imports for Microsoft Agent Framework
import os
import asyncio
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIResponsesClient

print("✅ Microsoft Agent Framework imports successful")
print("Using Microsoft Agent Framework (successor to AutoGen)")

# Check Llama Stack connectivity
import httpx

LLAMA_STACK_URL = "http://localhost:8321"

try:
    response = httpx.get(f"{LLAMA_STACK_URL}/v1/models")
    print(f"✅ Llama Stack is running at {LLAMA_STACK_URL}")
    print(f"Status: {response.status_code}")
except Exception as e:
    print(f"❌ Llama Stack not accessible: {e}")
    print("Make sure Llama Stack is running on port 8321")

✅ Microsoft Agent Framework imports successful
Using Microsoft Agent Framework (successor to AutoGen)
✅ Llama Stack is running at http://localhost:8321
Status: 200


## Configuration: Microsoft Agent Framework with Llama Stack

### How It Works

Microsoft Agent Framework uses **OpenAIResponsesClient** to connect to OpenAI-compatible servers like Llama Stack.

**Key Changes from AutoGen:**
- `OpenAIChatCompletionClient` → `OpenAIResponsesClient`
- Team-based architecture (similar to AutoGen v0.7.5)
- Async/await pattern for running tasks

In [20]:
# Create OpenAI Responses Client for Llama Stack
# Uses the /responses API (specialized for reasoning models)
chat_client = OpenAIResponsesClient(
    model_id="ollama/llama3.3:70b",  # Choose any other model of your choice
    api_key="not-needed",
    base_url="http://localhost:8321/v1"  # Llama Stack OpenAI-compatible endpoint
)

print("✅ Model client configured for Llama Stack")
print(f"Model: ollama/llama3.3:70b")
print(f"Base URL: http://localhost:8321/v1")
print(f"Client type: OpenAIResponsesClient")

✅ Model client configured for Llama Stack
Model: ollama/llama3.3:70b
Base URL: http://localhost:8321/v1
Client type: OpenAIResponsesClient


## Example 1: Simple Task with ChatAgent

### Pattern: Single Agent Task

Microsoft Agent Framework uses **ChatAgent** to create AI assistants powered by your model.

**ChatAgent Features:**
- Multi-turn by default (keeps calling tools until complete)
- Stateless (use `AgentThread` for conversation history)
- Configured with `instructions` (replaces AutoGen's `system_message`)
- Can be created directly or via client factory method

### Use Case: Solve a Math Problem

In [13]:
import asyncio

# Create a ChatAgent (replaces AutoGen's AssistantAgent)
# Method 1: Direct creation
assistant = ChatAgent(
    name="MathAssistant",
    chat_client=chat_client,
    instructions="You are a helpful AI assistant that solves math problems. Provide clear explanations and show your work."
)

# Method 2: Using client factory (more convenient)
# assistant = chat_client.create_agent(
#     name="MathAssistant",
#     instructions="You are a helpful AI assistant."
# )

print("✅ Agent created:", assistant.name)

# Define the task
task = "What is the sum of the first 10 prime numbers? Please calculate it step by step."

# Run the task (Agent Framework uses async)
# Note: ChatAgent is stateless - no conversation history between calls
result = await assistant.run(task)

print("\n" + "="*50)
print("Task Result:")
print(result.text if result.text else "No response")
print("="*50)

✅ Agent created: MathAssistant

Task Result:
To find the sum of the first 10 prime numbers, we need to follow these steps:

**Step 1: List out the first few prime numbers**
Prime numbers are numbers greater than 1 that have no divisors other than 1 and themselves. Let's start listing them out:
2, 3, 5, 7, 11, ...

**Step 2: Identify the first 10 prime numbers**
We need to find the next 4 prime numbers after 7 and 11.
13 is a prime number (only divisible by 1 and 13).
17 is a prime number (only divisible by 1 and 17).
19 is a prime number (only divisible by 1 and 19).
23 is a prime number (only divisible by 1 and 23).

So, the first 10 prime numbers are:
2, 3, 5, 7, 11, 13, 17, 19, 23, 29

**Step 3: Calculate the sum of these prime numbers**
Now, let's add them up:
2 + 3 = 5
5 + 5 = 10
10 + 7 = 17
17 + 11 = 28
28 + 13 = 41
41 + 17 = 58
58 + 19 = 77
77 + 23 = 100
100 + 29 = 129

**Step 4: Write the final answer**
The sum of the first 10 prime numbers is:
129

Therefore, the sum of the fi

## Example 2: Multi-Agent Team Collaboration

### Pattern: Sequential Workflow (Round-Robin Style)

Agent Framework uses **SequentialBuilder** to create workflows where agents take turns.
This replaces AutoGen's `RoundRobinGroupChat`.

**Key Concepts:**
- `SequentialBuilder`: Agents process messages sequentially
- Shared conversation history across all agents
- Each agent sees all previous messages

### Use Case: Write a Technical Blog Post

In [14]:
from agent_framework import SequentialBuilder, WorkflowOutputEvent

# Create specialist agents
researcher = ChatAgent(
    name="Researcher",
    chat_client=chat_client,
    instructions="You are a researcher. Provide accurate information, facts, and statistics about topics."
)

writer = ChatAgent(
    name="Writer",
    chat_client=chat_client,
    instructions="You are a technical writer. Write clear, engaging content based on research provided."
)

critic = ChatAgent(
    name="Critic",
    chat_client=chat_client,
    instructions="You are an editor. Review content for clarity, accuracy, and engagement. Suggest improvements."
)

print("✅ Team agents created: Researcher, Writer, Critic")

# Create a sequential workflow (round-robin collaboration)
# Each agent processes the input and builds on previous agents' work
workflow = SequentialBuilder().participants([researcher, writer, critic]).build()

task = """Write a 200-word blog post about the benefits of using Llama Stack for LLM applications.

Steps:
1. Researcher: Gather key information about Llama Stack
2. Writer: Create the blog post
3. Critic: Review and suggest improvements
"""

print("\n" + "="*50)
print("Running Sequential Workflow:")
print("="*50)

# Run the workflow and collect results
turn = 1
async for event in workflow.run_stream(task):
    if isinstance(event, WorkflowOutputEvent):
        # Final output contains full conversation history
        conversation_history = event.data
        for msg in conversation_history:
            print(f"\nTurn {turn} [{msg.role}]:")
            print(msg.text[:200] + "..." if len(msg.text or "") > 200 else msg.text)
            turn += 1

✅ Team agents created: Researcher, Writer, Critic

Running Sequential Workflow:

Turn 1 [user]:
Write a 200-word blog post about the benefits of using Llama Stack for LLM applications.

Steps:
1. Researcher: Gather key information about Llama Stack
2. Writer: Create the blog post
3. Critic: Revi...

Turn 2 [assistant]:
**Unlocking Efficient LLM Applications with Llama Stack**

The rise of Large Language Models (LLMs) has transformed the artificial intelligence landscape, enabling cutting-edge natural language proces...

Turn 3 [assistant]:


Turn 4 [assistant]:
 

---

**Critic's Review:**

The blog post effectively introduces the benefits of using Llama Stack for LLM applications, highlighting key advantages such as simplified model deployment and improved ...


## Example 3: Multi-Turn Conversations with AgentThread

### Pattern: Stateful Conversations

Unlike AutoGen, `ChatAgent` is **stateless by default**. To maintain conversation history across multiple interactions, use **AgentThread**.

**AgentThread Features:**
- Stores conversation history
- Allows context to carry across multiple `agent.run()` calls
- Can be backed by external storage (Redis, databases)

### Use Case: Interactive Analysis

In [15]:
# Create an analyst agent
analyst = ChatAgent(
    name="TechAnalyst",
    chat_client=chat_client,
    instructions="""You are a technical analyst. Analyze technical topics deeply:
    1. Break down complex concepts
    2. Identify pros and cons
    3. Provide recommendations
    """
)

print("✅ Analyst agent created")

# Create a new thread to maintain conversation state
thread = analyst.get_new_thread()

print("\n" + "="*50)
print("Multi-Turn Conversation with Thread:")
print("="*50)

# First interaction
result1 = await analyst.run(
    "Analyze the trade-offs between using local LLMs versus cloud-based APIs.",
    thread=thread
)
print("\n[Turn 1 - Initial Analysis]:")
print(result1.text[:400] + "..." if len(result1.text or "") > 400 else result1.text)

# Second interaction - builds on previous context
result2 = await analyst.run(
    "What about cost implications specifically?",
    thread=thread
)
print("\n[Turn 2 - Follow-up on Cost]:")
print(result2.text[:400] + "..." if len(result2.text or "") > 400 else result2.text)

# Third interaction - continues the conversation
result3 = await analyst.run(
    "Summarize your recommendation in one sentence.",
    thread=thread
)
print("\n[Turn 3 - Summary]:")
print(result3.text)

print("\n" + "="*50)
print(f"Thread maintained context across {3} turns")
print("="*50)

✅ Analyst agent created

Multi-Turn Conversation with Thread:

[Turn 1 - Initial Analysis]:
**Introduction**

Large Language Models (LLMs) have revolutionized the field of natural language processing, enabling applications such as text classification, sentiment analysis, and language translation. Two popular approaches to deploying LLMs are using local models and cloud-based APIs. In this analysis, we will break down the trade-offs between these two approaches, highlighting their pros an...

[Turn 2 - Follow-up on Cost]:
**Cost Implications: Local LLMs vs Cloud-based APIs**

The cost implications of using local LLMs versus cloud-based APIs are significant and can vary greatly depending on the specific requirements and deployment scenarios. Here's a detailed breakdown of the costs associated with each approach:

**Local LLMs**

1. **Initial Investment**:
	* Hardware: High-performance GPUs, high-capacity storage, an...

[Turn 3 - Summary]:
Organizations should choose between local LLMs a

## Example 4: Advanced Workflow with Custom Executors

### Pattern: Data-Flow Workflow with Code Review Loop

Agent Framework's **Workflow** enables complex orchestration using executors and edges.
Unlike AutoGen's event-driven model, workflows use **data-flow** architecture.

**Key Concepts:**
- `Executor`: Processing units (agents, functions, or sub-workflows)
- `WorkflowBuilder`: Build typed data-flow graphs
- `@executor` decorator: Define custom processing logic
- Edges route messages between executors

### Use Case: Iterative Code Review Until Approved

In [21]:
from agent_framework import WorkflowBuilder, executor, WorkflowContext, WorkflowOutputEvent
from typing_extensions import Never

# Create code review agents
code_developer = ChatAgent(
    name="Developer",
    chat_client=chat_client,
    instructions="""You are a developer. When you receive code review feedback:
    - Address ALL issues mentioned
    - Explain your changes
    - Present the improved code

    If no feedback is given, present your initial implementation."""
)

code_reviewer = ChatAgent(
    name="CodeReviewer",
    chat_client=chat_client,
    instructions="""You are a senior code reviewer. Review code for:
    - Bugs and edge cases
    - Performance issues
    - Security vulnerabilities
    - Best practices

    If the code looks good, say 'LGTM' (Looks Good To Me).
    If issues found, provide specific feedback for improvement."""
)

print("✅ Code review team created")

# Define custom executors for workflow
@executor(id="developer")
async def developer_executor(task: str, ctx: WorkflowContext[str]) -> None:
    """Developer creates or improves code based on input."""
    result = await code_developer.run(task)
    await ctx.send_message(result.text)

@executor(id="reviewer")
async def reviewer_executor(code: str, ctx: WorkflowContext[str, str]) -> None:
    """Reviewer checks code and either approves or requests changes."""
    result = await code_reviewer.run(f"Review this code:\n{code}")

    # Check if approved
    if "LGTM" in result.text or "looks good" in result.text.lower():
        await ctx.yield_output(f"✅ APPROVED\n\nCode:\n{code}\n\nReview:\n{result.text}")
    else:
        # Send feedback back to developer for revision
        await ctx.send_message(f"Feedback: {result.text}\n\nOriginal code:\n{code}", target_id="developer")

# Build workflow: developer → reviewer (with feedback loop)
workflow = (
    WorkflowBuilder()
    .add_edge(developer_executor, reviewer_executor)
    .add_edge(reviewer_executor, developer_executor)  # Feedback loop
    .set_start_executor(developer_executor)
    .build()
)

# task = "Implement a Python function to check if a string is a palindrome."
task = """Implement a Python function to validate email addresses with these requirements:
- Must have @ symbol
- Must have domain with at least one dot
- No spaces allowed
- Handle edge cases
- Include error messages
Make it production-ready with proper error handling."""


print("\n" + "="*50)
print("Code Review Workflow:")
print("="*50)

# Run workflow with streaming
iteration = 1
async for event in workflow.run_stream(task):
    if isinstance(event, WorkflowOutputEvent):
        print(f"\n[Iteration {iteration} - Final Result]:")
        print(event.data)
        iteration += 1

✅ Code review team created

Code Review Workflow:

[Iteration 1 - Final Result]:
✅ APPROVED

Code:
**Email Validation Function in Python**

The following Python function validates an email address, ensuring it meets the specified requirements.

### Implementation
```python
import re

def validate_email(email: str) -> bool:
    """
    Validates an email address.

    Args:
    - email (str): The email address to be validated.

    Returns:
    - bool: True if the email is valid, False otherwise.
    """

    # Define a regular expression pattern for email validation
    pattern = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"

    try:
        # Check if the input is a string
        if not isinstance(email, str):
            raise TypeError("Input must be a string.")

        # Remove leading and trailing whitespaces
        email = email.strip()

        # Check for spaces in the email address
        if " " in email:
            print("Error: No spaces are allowed in the email 

## Example 5: Concurrent Workflow Pattern

### Pattern: Parallel Processing

Agent Framework's **ConcurrentBuilder** enables parallel agent execution.
All agents process the input simultaneously and results are aggregated.

### Use Case: Multi-Perspective Analysis

In [25]:
from agent_framework import ConcurrentBuilder, WorkflowOutputEvent

# Create specialized analysts
technical_analyst = ChatAgent(
    name="TechnicalAnalyst",
    chat_client=chat_client,
    instructions="You analyze technical feasibility and implementation complexity."
)

business_analyst = ChatAgent(
    name="BusinessAnalyst",
    chat_client=chat_client,
    instructions="You analyze business value, ROI, and market impact."
)

security_analyst = ChatAgent(
    name="SecurityAnalyst",
    chat_client=chat_client,
    instructions="You analyze security implications, risks, and compliance."
)

print("✅ Analyst team created: Technical, Business, Security")

# Create concurrent workflow - all agents process in parallel
workflow = (
    ConcurrentBuilder()
    .participants([technical_analyst, business_analyst, security_analyst])
    .build()
)

# task = "Evaluate the proposal to deploy Llama Stack for our customer service chatbot."
task = "Evaluate the proposal to deploy a customer service chatbot."

print("\n" + "="*50)
print("Concurrent Analysis (Parallel Processing):")
print("="*50)

# Run workflow - agents work in parallel
async for event in workflow.run_stream(task):
    if isinstance(event, WorkflowOutputEvent):
        # Combined results from all agents
        results = event.data
        for i, result in enumerate(results, 1):
            print(f"\n[Analysis {i}]:")
            print(result.text[:1000] + "..." if len(result.text or "") > 1000 else result.text)
            print("-" * 50)

print("\n" + "="*50)
print("All agents completed in parallel")
print("="*50)

✅ Analyst team created: Technical, Business, Security

Concurrent Analysis (Parallel Processing):

[Analysis 1]:
Evaluate the proposal to deploy a customer service chatbot.
--------------------------------------------------

[Analysis 2]:
**Proposal Evaluation: Deploying a Customer Service Chatbot**

**Executive Summary:**
The proposal to deploy a customer service chatbot aims to enhance customer experience, reduce support queries, and optimize resource allocation. This evaluation assesses the technical feasibility and implementation complexity of the proposed solution.

**Technical Feasibility:**

1. **Natural Language Processing (NLP) Capabilities:** The chatbot's ability to understand and respond to customer inquiries accurately is crucial. Modern NLP libraries (e.g., NLTK, spaCy) and machine learning frameworks (e.g., TensorFlow, PyTorch) can support this requirement.
2. **Integration with Existing Systems:** Seamless integration with the CRM, helpdesk software, and other relevant 